In [27]:
!pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
!pip install prettytable

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
from mip import *
import math
import numpy as np
import random
from prettytable import PrettyTable
import time

In [30]:
start_time = time.time()

In [31]:
def scenario_generator(n_scenario, n_product, corn, wheat, sugerbeat):
  scenarios = np.zeros((n_scenario, n_product))
  probabilities = np.zeros(n_scenario)
  random_numbers = [random.uniform(0, 1) for i in range(n_scenario-1)]
  random_numbers.sort()

  for ind, prob in enumerate(random_numbers):

    if ind == 0: #SO we should consider the first random number before prob = 0
      probabilities[ind] = prob
      multiplier = prob/2

    elif ind == n_scenario - 2: #SO we should consider the last random number after prob = 1
      probabilities[ind] = (prob - random_numbers[ind-1])
      probabilities[ind+1] = (1 - prob)
      multiplier = (1 + prob)/2
      scenarios[ind+1][0] = corn[0] + (multiplier * (corn[1] - corn[0]))
      scenarios[ind+1][1] = wheat[0] + (multiplier * (wheat[1] - wheat[0]))
      scenarios[ind+1][2] = sugarbeat[0] + (multiplier * (sugarbeat[1] - sugarbeat[0]))
      multiplier = (prob + random_numbers[ind-1])/2  

    else: 
      probabilities[ind] = (prob - random_numbers[ind-1])
      multiplier = (prob + random_numbers[ind-1])/2

    scenarios[ind][0] = corn[0] + (multiplier * (corn[1] - corn[0]))
    scenarios[ind][1] = wheat[0] + (multiplier * (wheat[1] - wheat[0]))
    scenarios[ind][2] = sugarbeat[0] + (multiplier * (sugarbeat[1] - sugarbeat[0]))
  
  return scenarios, probabilities
    

In [32]:
def calculate_min_distance(remaining_scenarios, candidate_scenario, candidate_scenario_ind):

    min_i_distance = math.inf
    for ind, remained_scenario in enumerate(remaining_scenarios):
      if ind == candidate_scenario_ind:
        pass
      else:
        linear_distance = np.array(remained_scenario) - np.array(candidate_scenario)
        distance_i_j = np.sum(linear_distance**2)
        if distance_i_j <= min_i_distance:
          min_i_distance = distance_i_j
  
    return min_i_distance


In [33]:
def backward_elimination_step_0(scenarios, probabilities):
  deleted_scenarios = []
  remaining_scenarios = scenarios.copy()
  indices = [i for i in range(len(scenarios))]
  Z_min = math.inf
  candidate = None

  for ind1, candidate_scenario in enumerate(remaining_scenarios):
    min_i_distance = calculate_min_distance(remaining_scenarios, candidate_scenario, ind1)
    Z_i = probabilities[ind1]*min_i_distance
    if Z_i <= Z_min:
      Z_min = Z_i
      candidate = candidate_scenario
      candidate_ind = ind1
      candidate_prob = probabilities[ind1]
  
  return candidate, candidate_ind, candidate_prob


In [34]:
def backward_elimination(scenarios, probabilities):
  deleted_scenarios = []
  deleted_probabilities = []
  remaining_scenarios = scenarios.tolist()
  remaining_probabilities = probabilities.tolist()
  Z_min = math.inf
  candidate = None
  epsilon = 0.01

  first_deleted, first_deleted_ind, first_deleted_prob = backward_elimination_step_0(scenarios, probabilities)
  deleted_scenarios.append(first_deleted)
  deleted_probabilities.append(first_deleted_prob)
  del remaining_scenarios[first_deleted_ind]
  del remaining_probabilities[first_deleted_ind]


  while True:
    for ind, candidate_scenario in enumerate(remaining_scenarios):
      min_i_distance = calculate_min_distance(remaining_scenarios, candidate_scenario, ind)
      sum_probs = sum(deleted_probabilities[i] for i in range(len(deleted_scenarios))) + remaining_probabilities[ind]
      Z_i = min_i_distance * sum_probs
      if Z_i <= Z_min:
        Z_min = Z_i
        candidate = candidate_scenario
        candidate_ind = ind
        candidate_prob = remaining_probabilities[ind]
    
    if Z_min <= epsilon:
      deleted_scenarios.append(candidate)
      deleted_probabilities.append(candidate_prob)
      del remaining_scenarios[candidate_ind]
      del remaining_probabilities[candidate_ind]
      Z_min = math.inf
    else:
      break
    
  return remaining_scenarios, remaining_probabilities, deleted_scenarios, deleted_probabilities



In [35]:
def calculate_nearest_node(remaining_scenarios, deleted_scenario):
  min_i_distance = math.inf
  selected_node = None
  for ind, remained_scenario in enumerate(remaining_scenarios):
    linear_distance = np.array(remained_scenario) - np.array(deleted_scenario)
    distance_i_j = np.sum(linear_distance**2)
    if distance_i_j <= min_i_distance:
      min_i_distance = distance_i_j
      selected_node = ind
  return selected_node

In [36]:
def clustering_scenarios(remaining_scenarios, remaining_probabilities, deleted_scenarios, deleted_probabilities):

  for ind, deleted_scenario in enumerate(deleted_scenarios):
    nearest_node = calculate_nearest_node(remaining_scenarios, deleted_scenario)
    remaining_probabilities[nearest_node] += deleted_probabilities[ind]
  return remaining_probabilities


In [37]:
corn = [2.4, 3.6]
wheat = [2, 3]
sugarbeat = [16, 24]
n_scenario = 200
n_period = 3
first_year_scenarios, first_year_probs = scenario_generator(n_scenario, n_period, wheat, corn, sugarbeat)
first_remaining_scenarios, first_remaining_probs, first_deleted_scenarios, first_deleted_probs = backward_elimination(first_year_scenarios, first_year_probs)
second_year_scenarios, second_year_probs = scenario_generator(n_scenario, n_period, wheat, corn, sugarbeat)
second_remaining_scenarios, second_remaining_probs, second_deleted_scenarios, second_deleted_probs = backward_elimination(second_year_scenarios, second_year_probs)
first_remaining_probs = clustering_scenarios(first_remaining_scenarios, first_remaining_probs, first_deleted_scenarios, first_deleted_probs)
second_remaining_probs = clustering_scenarios(second_remaining_scenarios, second_remaining_probs, second_deleted_scenarios, second_deleted_probs)

In [38]:
#Data
n_first_scenarios = len(first_remaining_probs)
n_second_scenarios = len(second_remaining_probs)
n_period = 2
n_product = 3
n_product_to_buy = 2
n_product_to_sell = 4
n_planting_periods = 2

print(f'All generated scenarios: {len(first_year_scenarios)*len(second_year_scenarios)}')
print(f'Remaining scenarios after backward elimination: {n_first_scenarios * n_second_scenarios}')
print(f'It means we have {n_first_scenarios} scenarios in second year and {n_second_scenarios} in each scenario of second year in third year')

All generated scenarios: 40000
Remaining scenarios after backward elimination: 1295
It means we have 35 scenarios in second year and 37 in each scenario of second year in third year


In [39]:
m = Model('Farmaer')

p_c = [150, 230, 260]
b_c = [238, 210]
r = [200, 240]
s_p = [170, 150, 36, 10]

scenarios = [first_remaining_scenarios, second_remaining_scenarios]
probs = [first_remaining_probs, second_remaining_probs]
#########################
# Decision Variables
#########################
# Arcs to plant
x = [[m.add_var() for i in range(n_product)] for t in range(n_period)]
y = [[[m.add_var() for i in range(n_product_to_buy)] for s in range(len(scenarios[t]))] for t in range(n_period)]
w = [[[m.add_var() for i in range(n_product_to_sell)] for s in range(len(scenarios[t]))] for t in range(n_period)]
buying_cost = [[m.add_var() for s in range(len(scenarios[t]))] for t in range(n_period)]
revenue = [[m.add_var() for s in range(len(scenarios[t]))] for t in range(n_period)]

for t in range(n_period):
  m += sum(x[t][i] for i in range(n_product)) <= 500
  for s in range(len(scenarios[t])):
    m += scenarios[t][s][2]*x[t][2] >= w[t][s][2] + w[t][s][3]
    m += w[t][s][2] <= 6000
    m += buying_cost[t][s] == probs[t][s]*sum(b_c[j]*y[t][s][j] for j in range(n_product_to_buy))
    m += revenue[t][s] == probs[t][s]*sum(s_p[j]*w[t][s][j] for j in range(n_product_to_sell))
    for i in range(n_product_to_buy):
      m += scenarios[t][s][i]*x[t][i] + y[t][s][i] - w[t][s][i] >= r[i]

m += x[0][2] + x[1][2] <= 500

m.objective = minimize(sum(sum(p_c[i]*x[t][i] for i in range(n_product)) for t in range(n_period)) + sum(sum(buying_cost[t][s] for s in range(len(scenarios[t]))) for t in range(n_period)) - sum(sum(revenue[t][s] for s in range(len(scenarios[t]))) for t in range(n_period)))
m.optimize()
m.objective_value

-217317.37513444247

In [40]:
for t in range(n_period):
  print(f'Planting Area for wheat, corn and sugarbeat: {round(x[t][0].x, 3), round(x[t][1].x, 3), round(x[t][2].x, 3)}')

Planting Area for wheat, corn and sugarbeat: (166.676, 84.603, 248.721)
Planting Area for wheat, corn and sugarbeat: (163.778, 84.943, 251.279)


In [41]:
products = ['Wheat', 'Corn', 'Sugarbeat1', 'Sugarbeat2']
for i in range(n_first_scenarios):
  t = 0
  planting_arcs = [round(x[t][0].x, 3), round(x[t][1].x, 3), round(x[t][2].x, 3), 0]
  Need = [round(y[t][i][0].x, 3), round(y[t][i][1].x, 3), 0, 0]    
  sell = [round(w[t][i][0].x, 3), round(w[t][i][1].x, 3), round(w[t][i][2].x, 3), round(w[t][i][3].x, 3)]                                                          
  t = PrettyTable(['Product'] + products)
  t.add_row(['Planting Arcs'] + planting_arcs)
  t.add_row(['Need to buy'] + Need)
  t.add_row(['Sell Value'] + sell)
  print(t)

+---------------+---------+--------+------------+------------+
|    Product    |  Wheat  |  Corn  | Sugarbeat1 | Sugarbeat2 |
+---------------+---------+--------+------------+------------+
| Planting Arcs | 166.676 | 84.603 |  248.721   |     0      |
|  Need to buy  |   0.0   | 35.653 |     0      |     0      |
|   Sell Value  | 135.485 |  0.0   |  4005.006  |    0.0     |
+---------------+---------+--------+------------+------------+
+---------------+---------+--------+------------+------------+
|    Product    |  Wheat  |  Corn  | Sugarbeat1 | Sugarbeat2 |
+---------------+---------+--------+------------+------------+
| Planting Arcs | 166.676 | 84.603 |  248.721   |     0      |
|  Need to buy  |   0.0   | 33.573 |     0      |     0      |
|   Sell Value  |  138.9  |  0.0   |  4045.776  |    0.0     |
+---------------+---------+--------+------------+------------+
+---------------+---------+--------+------------+------------+
|    Product    |  Wheat  |  Corn  | Sugarbeat1 | Sugar

In [42]:
products = ['Wheat', 'Corn', 'Sugarbeat1', 'Sugarbeat2']
for i in range(n_second_scenarios):
  t = 1
  Need = [round(y[t][i][0].x, 3), round(y[t][i][1].x, 3), 0, 0]    
  sell = [round(w[t][i][0].x, 3), round(w[t][i][1].x, 3), round(w[t][i][2].x, 3), round(w[t][i][3].x, 3)]                                                          
  t = PrettyTable(['Product'] + products)
  t.add_row(['Need to buy'] + Need)
  t.add_row(['Sell Value'] + sell)
  print(t)

+-------------+---------+--------+------------+------------+
|   Product   |  Wheat  |  Corn  | Sugarbeat1 | Sugarbeat2 |
+-------------+---------+--------+------------+------------+
| Need to buy |   0.0   | 34.642 |     0      |     0      |
|  Sell Value | 129.958 |  0.0   |  4049.944  |    0.0     |
+-------------+---------+--------+------------+------------+
+-------------+---------+--------+------------+------------+
|   Product   |  Wheat  |  Corn  | Sugarbeat1 | Sugarbeat2 |
+-------------+---------+--------+------------+------------+
| Need to buy |   0.0   | 31.723 |     0      |     0      |
|  Sell Value | 134.648 |  0.0   |  4107.506  |    0.0     |
+-------------+---------+--------+------------+------------+
+-------------+---------+--------+------------+------------+
|   Product   |  Wheat  |  Corn  | Sugarbeat1 | Sugarbeat2 |
+-------------+---------+--------+------------+------------+
| Need to buy |   0.0   | 28.668 |     0      |     0      |
|  Sell Value | 139.557 

In [43]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 50.89424920082092 seconds ---
